In [5]:
import os
import betfairutil
import pandas as pd
import numpy as np
from string_grouper import match_strings, match_most_similar, \
	group_similar_strings, compute_pairwise_similarities, \
	StringGrouper
from unidecode import unidecode
from datetime import datetime

In [16]:
betfair_matches = pd.read_csv("DataFiles/betfair_matches.csv")
clegg_matches = pd.read_csv("DataFiles/clegg_matches_cleaned.csv").drop(columns=["WTA", "Unnamed: 0"])
ramirez_matches = pd.read_csv("DataFiles/ramirez_matches_cleaned.csv").drop(columns=["Unnamed: 0"])

In [49]:
ramirez_matches = ramirez_matches[ramirez_matches["date"] > "2016-07-01"].sort_values(by=["date", "player"]).reset_index(drop=True).sort_index(axis=1)
clegg_matches = clegg_matches[clegg_matches["date"] < "2020-02-17"].sort_values(by=["date", "player"]).reset_index(drop=True).sort_index(axis=1)

In [51]:
ramirez_matches

,date,inverse_avg,inverse_b365,inverse_best,match_id,outcome,player,rankdist,wikibuzz,year
0,2016-07-02,0.862069,0.862069,0.833333,13015,1,Agnieszka Radwanska,-0.324561,-2.170413,2016.0
1,2016-07-02,0.190476,0.181818,0.173913,13013,0,Alize Cornet,0.094718,-1.841631,2016.0
2,2016-07-02,0.909091,0.909091,0.877193,13012,1,Angelique Kerber,-0.240826,-0.050116,2016.0
3,2016-07-02,0.446429,0.434783,0.416667,13010,0,Anna-Lena Friedsam,0.002864,-0.147916,2016.0
4,2016-07-02,0.147710,0.142857,0.129534,13012,0,Carina Witthoeft,0.240826,0.050116,2016.0
...,...,...,...,...,...,...,...,...,...,...
16161,2020-02-15,0.657895,0.666667,0.628931,4141,1,Magda Linette,-0.014286,0.517943,2020.0
16162,2020-02-15,0.431034,0.421941,0.408163,4142,0,Maria Sakkari,-0.007619,-1.612937,2020.0
16163,2020-02-15,0.396825,0.381679,0.378788,4141,0,Patricia Maria Tig,0.014286,-0.517943,2020.0
16164,2020-02-16,0.414938,0.421941,0.386100,4143,0,Elena Rybakina,0.085000,2.935837,2020.0


In [50]:
clegg_matches

,date,inverse_avg,inverse_b365,inverse_best,match_id,outcome,player,rankdist,wikibuzz,year
0,2016-07-02,0.862069,0.862069,0.833333,1552,1,Agnieszka Radwanska,-0.324561,-1.764948,2016
1,2016-07-02,0.190476,0.181818,0.173913,1550,0,Alize Cornet,0.094718,-1.782315,2016
2,2016-07-02,0.909091,0.909091,0.877193,1546,1,Angelique Kerber,-0.240826,-0.047462,2016
3,2016-07-02,0.446429,0.434783,0.416667,1545,0,Anna-Lena Friedsam,0.002864,-0.157395,2016
4,2016-07-02,0.147710,0.142857,0.129534,1546,0,Carina Witthoeft,0.240826,0.047462,2016
...,...,...,...,...,...,...,...,...,...,...
15797,2020-02-15,0.396825,0.381679,0.378788,10269,0,Patricia Maria Tig,0.014286,-1.220059,2020
15798,2020-02-16,0.414938,0.421941,0.386100,10298,0,Elena Rybakina,0.085000,2.931204,2020
15799,2020-02-16,0.641026,0.636943,0.621118,10298,1,Kiki Bertens,-0.085000,-2.931204,2020
15800,2020-02-16,0.282486,0.285714,0.244499,10271,0,Leonie Kung,0.020276,-0.866364,2020


In [61]:
df_all = clegg_matches.merge(ramirez_matches.drop_duplicates(), on=["date", "player"],
                   how='left', indicator=True)

In [62]:
df_all[df_all["_merge"] == "left_only"][["date", "player", "_merge"]]

,date,player,_merge
76,2016-07-12,Ioana Minca,left_only
85,2016-07-12,Patricia Maria Tig,left_only
87,2016-07-12,Polina Leykina,left_only
91,2016-07-12,Shilin Xu,left_only
106,2016-07-14,Danka Kovinic,left_only
...,...,...,...
16163,2020-02-14,NaN,left_only
16167,2020-02-15,Leonie Kung,left_only
16170,2020-02-15,Nao Hibino,left_only
16174,2020-02-16,Leonie Kung,left_only
